In [13]:
# 以下代码的目的在于解决【中文字符】问题
import sys
stdout = sys.stdout
# 新版sys, 需要先reload(sys)再进行sys.setdefaultencoding操作
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout
import os
os.getcwd()
os.chdir('/Users/Max/PycharmProjects/user_churn_second')
print '1'

1


In [7]:
# 常用module
import pandas as pd 
import numpy as np
from imp import reload
from sklearn.externals import joblib

# machine learning
from sklearn.metrics import recall_score
def my_custom_scorer(y_true, y_pred):
    return recall_score(y_true, y_pred, average='micro')

# local
import data_extract_X as dx
import data_extract_battle as db
import data_clean as dc

### 案例：提取4月13日新用户为例，验证模型的可靠性

 1、提取用户y标签
 
 2、提取用户X标签
 
 3、处理X数据
 
 4、导入模型做预测
 
 5、给出预测用户名单

注意：指标为3日留存

#### 1、提取y标签

In [8]:
reload(db)
# initialisaiton
date_start = '2018-04-13'
date_end = '2018-04-13'
db_class = db.data_extract_class(date_start, date_end)
## operations
db_class.download_new_userid()
y = db_class.read_battle_y(retained_day=3).drop_duplicates(keep='first')

数据库连接完成！
数据提取中......
数据提取完成，耗时0.113207101822



all userid have downloaded!
数据库连接完成！
数据提取中......


数据提取完成，耗时0.312450885773



In [4]:
y.head() # 提取完成，结果如下 - 1为留存，0为流失

,full_id,y
0,5ad07d91b31aaa7021bc00e5,0
1,5acfffc36077d1151e9a6d71,0
2,5ad0584aecfb9e2e3a182fb3,0
3,5ad07c835fabdf47f013d94a,0
4,5ad0d1e2ecfb9e2e3a33d8d0,0


#### 2、提取X标签

In [9]:
reload(dx)
date_start = '2018-04-13'
date_end = '2018-04-13'
dx_X = dx.create_new_X(date_start, date_end)
result = dx_X.main(date_start)
result.to_csv('output/demo_X.csv', encoding='utf-8')

数据库连接完成！
数据提取中......


数据提取完成，耗时0.319646120071



数据库连接完成！
数据提取中......


数据提取完成，耗时0.261146068573



数据库连接完成！
数据提取中......


...

数据提取完成，耗时0.324486017227



数据库连接完成！
数据提取中......


数据提取完成，耗时0.490743875504



数据库连接完成！
数据提取中......


数据提取完成，耗时0.297152042389



In [6]:
# 结果如下
result.head()

,userid,os,city,gender,status,completed_info,SignIn,date,weekend,fromchannel,...,share,favorite,read,actions,path1,path2,path3,path4,path5,path6
0,5ad07d91b31aaa7021bc00e5,Android,长春,female,worker,0,1.0,2018-04-13,0,Oppo,...,0.0,0.0,13.0,文章详情页文章详情页文章详情页文章详情页文章详情页文章详情页文章详情页文章详情页买手主页文章...,0.0,0.0,0.0,0.0,0.0,0.0
1,5acfffc36077d1151e9a6d71,None,None,male,worker,0,0.0,2018-04-13,0,None,...,0.0,0.0,21.0,首页文章详情页首页文章详情页首页文章详情页首页文章详情页首页文章详情页首页文章详情页首页文章...,0.0,0.0,0.0,0.0,0.0,0.0
2,5ad0584aecfb9e2e3a182fb3,iOS,西安,female,worker,0,0.0,2018-04-13,0,None,...,0.0,0.0,0.0,首页首页发现买手发现买手首页,0.0,0.0,0.0,0.0,0.0,0.0
3,5ad07c835fabdf47f013d94a,None,None,male,worker,0,0.0,2018-04-13,0,None,...,0.0,0.0,3.0,首页文章详情页文章详情页首页发现买手发现买手买手主页文章详情页买手主页文章详情页买手主页文章...,0.0,0.0,0.0,0.0,0.0,0.0
4,5ad0d1e2ecfb9e2e3a33d8d0,None,None,female,worker,0,0.0,2018-04-13,0,None,...,0.0,0.0,0.0,首页发现买手发现买手发现买手发现买手,0.0,0.0,0.0,0.0,0.0,0.0


#### 3、预处理X

In [10]:
reload(dc)
# And now to load scaler
X = result.copy().drop(['date', 'actions', 'path3', 'path4', 'path5', 'path6'], axis=1)
def preprocessing_df(X, y):
    
    X_y = pd.merge(X, y, left_on='userid', right_on='full_id', how='left')
    dc_dc = dc.data_clean()
    X_y = dc_dc.quick_correct(X_y)

    # 异常值处理
    # kick outlier out
    # 1. outlier 
    X_y = dc_dc.outlier_deal(X_y)

    # 得到 dummies 变量
    y_fea = X_y.loc[:, 'y']
    X_fea = pd.get_dummies(X_y.drop(['y', 'userid', 'full_id'], axis=1))

    # scale  
    # 归一化
    model_columns = np.loadtxt('output/model_columns', dtype='S')
    scaler = joblib.load('output/scaler.save') 
    X_train_scaler = scaler.transform(X_fea[model_columns])    
    return X_train_scaler, y_fea
X_train_scaler, y_fea = preprocessing_df(X, y)

In [11]:
# 恢复模型并做预测
# set score function
recoverd_model = joblib.load("output/train_model.m")
recoverd_model.score(X_train_scaler, y_fea)

0.78727841501564133

In [12]:
# 预测
recoverd_model.predict(X_train_scaler)

array([0, 0, 0, ..., 0, 0, 0])